# evaluation

In [2]:
import torch
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn

# read data


## model

In [3]:
# Model definition (should match the training script)
class ANNModel(nn.Module):
    def __init__(self, input_size):
        super(ANNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.output(x))
        return x

In [5]:
model_path = '../models/'
data_path = '../data/tensors/'

# Load the saved model and parameters
checkpoint = torch.load(model_path + 'model_checkpoint.pth')

input_size = checkpoint['input_size']
model = ANNModel(input_size=input_size)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Load normalization parameters if needed
mean = checkpoint.get('mean')
std = checkpoint.get('std')

# Prepare the test data (already preprocessed and normalized)
X_test_tensor, y_test_tensor = torch.load(data_path + 'test_data.pt')
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Evaluation
y_pred = []
y_true = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        preds = (outputs > 0.5).float()  # Threshold to binarize output
        
        y_pred.extend(preds.cpu().numpy())
        y_true.extend(y_batch.cpu().numpy())

# Generate and print classification metrics
print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

         0.0       0.87      0.97      0.92       520
         1.0       0.97      0.86      0.91       545

    accuracy                           0.91      1065
   macro avg       0.92      0.92      0.91      1065
weighted avg       0.92      0.91      0.91      1065



C:\Users\reza\AppData\Local\Temp\ipykernel_37320\1554190881.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path + 'model_checkpoint.pth')
